In [1]:
pip list

Package                       Version
----------------------------- ---------------
aiobotocore                   2.4.2
aiofiles                      22.1.0
aiohttp                       3.8.3
aioitertools                  0.7.1
aiosignal                     1.2.0
aiosqlite                     0.18.0
alabaster                     0.7.12
anaconda-catalogs             0.2.0
anaconda-client               1.12.0
anaconda-navigator            2.4.2
anaconda-project              0.11.1
anyio                         3.5.0
appdirs                       1.4.4
argon2-cffi                   21.3.0
argon2-cffi-bindings          21.2.0
arrow                         1.2.3
astroid                       2.14.2
astropy                       5.1
asttokens                     2.0.5
async-timeout                 4.0.2
atomicwrites                  1.4.0
attrs                         22.1.0
Automat                       20.2.0
autopep8                      1.6.0
Babel                         2.11.0
backcal

In [2]:
n = 5
print(n)

5


In [1]:
pip install streamlit


  Obtaining dependency information for streamlit from https://files.pythonhosted.org/packages/50/28/fdfe9711d553cfaf217cafc9b8ba21479c663b6504cc96652465b153e04b/streamlit-1.26.0-py2.py3-none-any.whl.metadata
  Obtaining dependency information for altair<6,>=4.0 from https://files.pythonhosted.org/packages/f2/b4/02a0221bd1da91f6e6acdf0525528db24b4b326a670a9048da474dfe0667/altair-5.1.1-py3-none-any.whl.metadata
  Using cached blinker-1.6.2-py3-none-any.whl (13 kB)
  Obtaining dependency information for cachetools<6,>=4.0 from https://files.pythonhosted.org/packages/a9/c9/c8a7710f2cedcb1db9224fdd4d8307c9e48cbddc46c18b515fefc0f1abbe/cachetools-5.3.1-py3-none-any.whl.metadata
  Obtaining dependency information for click<9,>=7.0 from https://files.pythonhosted.org/packages/00/2e/d53fa4befbf2cfa713304affc7ca780ce4fc1fd8710527771b58311a3229/click-8.1.7-py3-none-any.whl.metadata
  Obtaining dependency information for importlib-metadata<7,>=1.4 from https://files.pythonhosted.org/packages/cc/37

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [2]:
pip list

Package                   VersionNote: you may need to restart the kernel to use updated packages.

------------------------- ------------
altair                    5.1.1
asttokens                 2.4.0
attrs                     23.1.0
backcall                  0.2.0
blinker                   1.6.2
cachetools                5.3.1
certifi                   2023.7.22
charset-normalizer        3.2.0
click                     8.1.7
colorama                  0.4.6
comm                      0.1.4
debugpy                   1.8.0
decorator                 5.1.1
executing                 1.2.0
gitdb                     4.0.10
GitPython                 3.1.36
idna                      3.4
importlib-metadata        6.8.0
ipykernel                 6.25.2
ipython                   8.15.0
jedi                      0.19.0
Jinja2                    3.1.2
jsonschema                4.19.0
jsonschema-specifications 2023.7.1
jupyter_client            8.3.1
jupyter_core              5.3.1
markdown-it-py   

In [28]:
import pandas 
import folium 
import openpyxl

In [43]:
# Utilisez pd.read_excel() pour lire le fichier Excel dans un DataFrame
data = pandas.read_excel("accidentsVelo.xlsx", decimal=",")  
data.head(5)

,Num_Acc,date,an,mois,jour,hrmn,dep,com,lat,long,...,secuexist,equipement,obs,obsm,choc,manv,vehiculeid,typevehicules,manoeuvehicules,numVehicules
0,200500000030,2005-01-13,2005,janvier,jeudi,19:45:00,62,62331,50.300,2.840,...,0,Aucun équipement,Sans objet,Véhicule,Côté gauche,Changeant de file à gauche,200500000030B02,Transport en commun,Dépassant à gauche,1.0
1,200500000034,2005-01-19,2005,janvier,mercredi,10:45:00,62,62022,0.000,0.000,...,0,Aucun équipement,Sans objet,Véhicule,Avant,Sans changement de direction,200500000034B02,"VU seul 1,5T <= PAC <= avec ou sans remorque",Tournant à gauche,1.0
2,200500000078,2005-01-26,2005,janvier,mercredi,13:15:00,2,2173,0.000,0.000,...,Oui,Casque,Sans objet,Véhicule,Avant,Sans changement de direction,200500000078B02,Vl seul,Tournant à gauche,1.0
3,200500000093,2005-01-03,2005,janvier,lundi,13:30:00,2,2810,49.255,3.094,...,0,Aucun équipement,Sans objet,Véhicule,Avant gauche,Manoeuvre d'évitement,200500000093B02,Vl seul,Manoeuvre d'évitement,1.0
4,200500000170,2005-01-29,2005,janvier,samedi,18:30:00,76,76196,0.000,0.000,...,Oui,Autre,Sans objet,Véhicule,Arrière,"Même sens, même file",200500000170A01,"VU seul 1,5T <= PAC <= avec ou sans remorque","Même sens, même file",1.0


In [44]:
data=data.dropna(subset=['long'])
data=data.dropna(subset=['lat'])

In [52]:
#réduction de la table pour voir (janvier 2021)
data2021 = data[data.an==2021]
data2021 = data2021[data2021.mois == "janvier"]
data2021.head

<bound method NDFrame.head of             Num_Acc       date    an     mois      jour      hrmn dep    com  \
73091  202100029069 2021-01-29  2021  janvier  vendredi  10:28:00   6   6029   
74363  202100048196 2021-01-07  2021  janvier     jeudi  17:30:00  88  88481   
74461  202100049789 2021-01-31  2021  janvier  dimanche  13:18:00  31  31082   
74611  202100052718 2021-01-29  2021  janvier  vendredi  16:30:00  42  42315   
74612  202100052718 2021-01-29  2021  janvier  vendredi  16:30:00  42  42315   
...             ...        ...   ...      ...       ...       ...  ..    ...   
74753  202100056317 2021-01-02  2021  janvier    samedi  18:30:00  44  44168   
74754  202100056362 2021-01-04  2021  janvier     lundi  08:20:00  64  64138   
74755  202100056404 2021-01-01  2021  janvier  vendredi  16:55:00  54  54395   
74756  202100056424 2021-01-02  2021  janvier    samedi  15:40:00  75  75110   
74757  202100056508 2021-01-01  2021  janvier  vendredi  12:20:00  64  64400   

         

In [55]:
# Création de la carte centrée sur la France
m = folium.Map(location=[46.603354, 1.888334], zoom_start=6)                                                              

# Parcourir les lignes du DataFrame pour ajouter des marqueurs sur la carte
for index, row in data2021.iterrows():
    latitude, longitude = row['lat'], row['long']
    gravite = row['grav']

    # Détermination de la couleur du marqueur en fonction de la gravité
    if gravite == 'Tué':
        marker_color = 'red'
    elif gravite == 'Indemne':
        marker_color = 'green'
    elif gravite == 'Blessé léger':
        marker_color = 'orange'
    elif gravite == 'Blessé hospitalisé':
        marker_color = 'blue'

    # Ajout d'un marqueur à la carte
    folium.Marker([latitude, longitude], icon=folium.Icon(color=marker_color)).add_to(m)
    
# Enregistrement de la carte dans un fichier HTML
m.save('carte_accidents_velo.html')

# Ouvrir la carte dans le navigateur par défaut
import webbrowser
webbrowser.open('carte_accidents_velo.html')

True